In [265]:
import pandas as pd
import numpy as np

In [266]:
df_micro = pd.DataFrame(
    {
        "ni_transito": [
            10000,
            100000,
            1000000,
            10000000,
            100000000,
            1000000000,
        ]
    }
)

In [267]:
df_micro

,ni_transito
0,10000
1,100000
2,1000000
3,10000000
4,100000000
5,1000000000


# FORMULAS FIJAS DEL MODELO

In [268]:
modulo_rigidez_MPa = 4500
espesor_mezcla_asfaltica_cm = 15
numero_ciclos_carga = 134

In [269]:
rigidez_convertida = 145 * modulo_rigidez_MPa
rigidez_convertida


652500

In [270]:
espesor_mezcla_pulgadas = espesor_mezcla_asfaltica_cm / 2.54
espesor_mezcla_asfaltica_cm

15

In [271]:
factor_ajuste_modelo = 10 ** (4.84 * (11 / (11 + 5) - 0.69))
factor_ajuste_modelo

0.9725232662363479

In [272]:
factor_ajuste_resistencia = float(1 / (0.000398 + (0.003602 / (1 + np.exp(11.02 - 3.49 * espesor_mezcla_pulgadas)))))
factor_ajuste_resistencia

250.01539692873743

# Tabla de Calculo de indices de fatiga - tabla derecha

In [273]:
# COPIA DF micro a  calculos base fatiga
df_calculos_base_fatiga = df_micro.copy()

In [274]:
df_calculos_base_fatiga

,ni_transito
0,10000
1,100000
2,1000000
3,10000000
4,100000000
5,1000000000


In [275]:
# Calculo SPDM DERECHO 
df_calculos_base_fatiga['SPDM'] = (1000000 * (df_calculos_base_fatiga['ni_transito'] * 1/10 * (0.856 * 11 + 1.08)**-5 * (modulo_rigidez_MPa * 1000000)**1.8)**(-1/5))

In [276]:
# Calculo de AI MS-1
df_calculos_base_fatiga['AI MS-1'] = 1000000 * ((df_calculos_base_fatiga['ni_transito'] * rigidez_convertida**0.854) / (18.4 * 0.00432 * factor_ajuste_modelo))**(-1 / 3.291)

In [277]:
# Calculo de Cedex-Shell
df_calculos_base_fatiga['Cedex-Shell'] = 1000000 * (df_calculos_base_fatiga['ni_transito'] / 1.02e-13)**-0.2

In [278]:
# Calculo de Cedex-COST
df_calculos_base_fatiga['Cedex-COST'] = 1000000 * (df_calculos_base_fatiga['ni_transito'] / 0.00000000906)**(-1 / 3.6706)

In [279]:
# Calculo de Illinois
df_calculos_base_fatiga['Illinois'] = 1000000 * (df_calculos_base_fatiga['ni_transito'] / 0.000005)**(-1 / 3)

In [280]:
# Calculo de Minnesota
df_calculos_base_fatiga['Minnesota'] = 1000000 * (df_calculos_base_fatiga['ni_transito'] / 0.00000283)**(-1 / 3.206)

In [281]:
# Calculo de USACE
df_calculos_base_fatiga['USACE'] = 1000000 * (df_calculos_base_fatiga['ni_transito'] / 478.63 * (145 * modulo_rigidez_MPa)**2.66)**(-1 / 5)

In [282]:
#Calculo de ME-PDG
df_calculos_base_fatiga['ME-PDG'] = 1000000 * (df_calculos_base_fatiga['ni_transito'] * rigidez_convertida**1.281 / (18.4 * 0.00432 * factor_ajuste_modelo * 0.007566 * factor_ajuste_resistencia))**(-1 / 3.9492)

In [283]:
# #Calculo de AUSTROADS
df_calculos_base_fatiga['AUSTROADS'] = ((df_calculos_base_fatiga['ni_transito'] / 2)**-0.2 * 6918 * (0.856 * 11 + 1.08) / (modulo_rigidez_MPa**0.36))

In [284]:
df_calculos_base_fatiga

,ni_transito,SPDM,AI MS-1,Cedex-Shell,Cedex-COST,Illinois,Minnesota,USACE,ME-PDG,AUSTROADS
0,10000,882.810149,866.714311,399.687011,523.720928,793.700526,1051.398728,439.180601,775.599737,639.814199
1,100000,557.015547,430.543422,252.185455,279.682250,368.403150,512.686980,277.104226,432.934118,403.695468
2,1000000,351.453050,213.873979,159.118265,149.358478,170.997595,249.998342,174.840946,241.660668,254.714620
3,10000000,221.751883,106.242661,100.396838,79.761783,79.370053,121.905127,110.317179,134.893223,160.714061
4,100000000,139.915979,52.776420,63.346122,42.595119,36.840315,59.443834,69.605434,75.296413,101.403717
5,1000000000,88.281015,26.216875,39.968701,22.747036,17.099759,28.986225,43.918060,42.029909,63.981420


# Tabla de Calculo de indices de fatiga - tabla izquierda

In [285]:
# Crear un data frame vacio
df_resultados_fatiga = pd.DataFrame()

In [286]:
#Calcular SPDM

df_resultados_fatiga["SPDM"] = 10 * (0.856 * 11 + 1.08)**5 * (modulo_rigidez_MPa * 1000000)**-1.8 * (df_calculos_base_fatiga['SPDM'] / 1000000)**-5
df_resultados_fatiga

,SPDM
0,1.000000e+04
1,1.000000e+05
2,1.000000e+06
3,1.000000e+07
4,1.000000e+08
5,1.000000e+09


In [287]:
#Calcular AI MS-1

df_resultados_fatiga["AI MS-1"] = 18.4 * 0.00432 * factor_ajuste_modelo * (df_calculos_base_fatiga['AI MS-1'] / 1000000)**-3.291 * rigidez_convertida**-0.854
df_resultados_fatiga

,SPDM,AI MS-1
0,1.000000e+04,1.000000e+04
1,1.000000e+05,1.000000e+05
2,1.000000e+06,1.000000e+06
3,1.000000e+07,1.000000e+07
4,1.000000e+08,1.000000e+08
5,1.000000e+09,1.000000e+09


In [288]:
#Calcular Cedex-Shell - Error

df_resultados_fatiga["Cedex-Shell"] = 0.000000000000102 * (df_calculos_base_fatiga['Cedex-Shell'] / 1000000)**-5
df_resultados_fatiga

,SPDM,AI MS-1,Cedex-Shell
0,1.000000e+04,1.000000e+04,1.000000e+04
1,1.000000e+05,1.000000e+05,1.000000e+05
2,1.000000e+06,1.000000e+06,1.000000e+06
3,1.000000e+07,1.000000e+07,1.000000e+07
4,1.000000e+08,1.000000e+08,1.000000e+08
5,1.000000e+09,1.000000e+09,1.000000e+09


In [290]:
#Calcular Cedex-COST

df_resultados_fatiga["Cedex-COST"] = 0.00000000906 * (df_calculos_base_fatiga['Cedex-COST'] / 1000000)**-3.6706
df_resultados_fatiga

,SPDM,AI MS-1,Cedex-Shell,Cedex-COST
0,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
1,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05
2,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06
3,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
4,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08
5,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09


In [291]:
#Calcular Illinois

df_resultados_fatiga["Illinois"] = 0.000005 * (df_calculos_base_fatiga['Illinois'] / 1000000)**-3
df_resultados_fatiga

,SPDM,AI MS-1,Cedex-Shell,Cedex-COST,Illinois
0,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
1,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05
2,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06
3,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
4,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08
5,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09


In [294]:
#Calcular Minnesota

df_resultados_fatiga["Minnesota"] = 0.00000283 * (df_calculos_base_fatiga['Minnesota'] / 1000000)** -3.206
df_resultados_fatiga

,SPDM,AI MS-1,Cedex-Shell,Cedex-COST,Illinois,Minnesota
0,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
1,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05
2,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06
3,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
4,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08
5,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09


In [295]:
#Calcular USACE

df_resultados_fatiga["USACE"] = 478.63 * (df_calculos_base_fatiga['USACE'] / 1000000)**-5 * (145 * modulo_rigidez_MPa)**-2.66
df_resultados_fatiga

,SPDM,AI MS-1,Cedex-Shell,Cedex-COST,Illinois,Minnesota,USACE
0,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
1,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05
2,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06
3,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
4,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08
5,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09


In [297]:
#Calcular ME-PDG

df_resultados_fatiga["ME-PDG"] = 18.4 * 0.00432 * 0.007566 * factor_ajuste_resistencia * factor_ajuste_modelo * (df_calculos_base_fatiga['ME-PDG'] / 1000000)**-3.9492 * (rigidez_convertida)**-1.281
df_resultados_fatiga

,SPDM,AI MS-1,Cedex-Shell,Cedex-COST,Illinois,Minnesota,USACE,ME-PDG
0,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
1,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05
2,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06
3,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
4,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08
5,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09


In [298]:
#Calcular AUSTROADS

df_resultados_fatiga["AUSTROADS"] = 2 * (6918 * (0.856 * 11 + 1.08) / (modulo_rigidez_MPa**0.36 * df_calculos_base_fatiga['AUSTROADS']))**5
df_resultados_fatiga

,SPDM,AI MS-1,Cedex-Shell,Cedex-COST,Illinois,Minnesota,USACE,ME-PDG,AUSTROADS
0,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
1,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05
2,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06
3,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
4,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08
5,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09
